In [1]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [3]:
XMIN = 1470
XMAX = 2840
YMIN = 50
YMAX = 2000

In [4]:
def find_centroids(th, min_size):
    if min_size < 1:
        raise ValueError('min_size must be at least 1')
    
    _, cont, _ = cv2.findContours(th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    M = map(cv2.moments, cont)
    
    cxs = []
    cys = []

    for i, m in enumerate(M):
        if m['m00'] < min_size:
            continue

        cx = int(m['m10']/m['m00'])
        cy = int(m['m01']/m['m00'])

        cxs.append(cx)
        cys.append(cy)
      
    return cxs, cys

In [5]:
def get_masked_window(grayed, cx, cy, size):
    windowed = grayed[cy-size//2:cy+size//2, cx-size//2:cx+size//2]
    _, bw = cv2.threshold(windowed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, cont, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if 0 < len(cont) < 3:
        return windowed * (bw == 0)
    else:
        return None

## TF model

In [8]:
video = cv2.VideoCapture('videos/TandemRun.mp4')

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# CNN
SIZE = 28

tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, SIZE*SIZE])
y_ = tf.placeholder(tf.float32, [None, 3])

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, SIZE, SIZE, 1])
    
with tf.name_scope('conv'):
    W_conv = weight_variable([3, 3, 1, 8])
    b_conv = bias_variable([8])
    h_conv = tf.nn.relu(tf.nn.conv2d(x_image, W_conv, strides=[1, 1, 1, 1], padding='SAME'))
    
with tf.name_scope('pool'):
    h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('fc1'):
    W_fc1 = weight_variable([SIZE//2*SIZE//2*8, 100])
    b_fc1 = bias_variable([100])
        
    h_pool_flat = tf.reshape(h_pool, [-1, SIZE//2*SIZE//2*8])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
    
with tf.name_scope('fc2'):
    W_fc2 = weight_variable([100, 3])
    b_fc2 = bias_variable([3])

    y = tf.matmul(h_fc1, W_fc2) + b_fc2
    
saver = tf.train.Saver()

sess = tf.InteractiveSession()
saver.restore(sess, 'tf_save/trained_model.ckpt')

INFO:tensorflow:Restoring parameters from tf_save/trained_model.ckpt


In [10]:
MIN_CONTOUR_SIZE = 45
WINDOW_SIZE = SIZE
frame_count = 0

out_video = cv2.VideoWriter('out.mp4', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (3840, 2160))

start_time = time.time()

while video.isOpened():
    ret, frame = video.read()

    frame_count += 1
    if frame_count % 30 == 0:
        print('Processed {}s of video. Time Elapsed {}s'.format(frame_count//30, time.time()-start_time))
        start_time = time.time()
        if frame_count // 30 > 30:
            break
      
    cropped = frame[YMIN:YMAX, XMIN:XMAX]
    grayed = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(grayed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
    cxs, cys = find_centroids(th, MIN_CONTOUR_SIZE)
    
    for cx, cy in zip(cxs, cys):
        masked = get_masked_window(grayed, cx, cy, WINDOW_SIZE)
        # If image patch is a transport
        if masked is not None and masked.size == WINDOW_SIZE * WINDOW_SIZE:
            if sess.run(tf.argmax(y, 1), feed_dict={x: [masked.flatten()]}) == 1:
                print('Tandem found in frame {}.'.format(frame_count))
                cv2.rectangle(frame, (cx+XMIN-15, cy+YMIN-15), (cx+XMIN+15, cy+YMIN+15), (0, 255, 0), 3)
       
    out_video.write(frame)

out_video.release()

Tandem found in frame 1.
Tandem found in frame 2.
Tandem found in frame 3.
Tandem found in frame 4.
Tandem found in frame 11.
Tandem found in frame 12.
Tandem found in frame 13.
Tandem found in frame 15.
Tandem found in frame 20.
Processed 1s of video. Time Elapsed 10.795089960098267s
Tandem found in frame 31.
Tandem found in frame 32.
Tandem found in frame 36.
Tandem found in frame 37.
Tandem found in frame 52.
Tandem found in frame 52.
Tandem found in frame 53.
Tandem found in frame 54.
Tandem found in frame 55.
Processed 2s of video. Time Elapsed 14.597672939300537s
Tandem found in frame 66.
Tandem found in frame 67.
Tandem found in frame 70.
Tandem found in frame 78.
Tandem found in frame 81.
Tandem found in frame 82.
Tandem found in frame 86.
Tandem found in frame 87.
Tandem found in frame 88.
Tandem found in frame 89.
Processed 3s of video. Time Elapsed 18.849162817001343s
Tandem found in frame 90.
Tandem found in frame 91.
Tandem found in frame 94.
Tandem found in frame 95.
Tand

Tandem found in frame 398.
Tandem found in frame 399.
Tandem found in frame 400.
Tandem found in frame 400.
Tandem found in frame 401.
Tandem found in frame 402.
Tandem found in frame 403.
Tandem found in frame 404.
Tandem found in frame 405.
Tandem found in frame 406.
Tandem found in frame 406.
Tandem found in frame 407.
Tandem found in frame 408.
Tandem found in frame 408.
Tandem found in frame 409.
Tandem found in frame 409.
Tandem found in frame 410.
Tandem found in frame 411.
Tandem found in frame 412.
Tandem found in frame 417.
Processed 14s of video. Time Elapsed 67.97847294807434s
Tandem found in frame 424.
Tandem found in frame 426.
Tandem found in frame 427.
Tandem found in frame 433.
Tandem found in frame 434.
Tandem found in frame 435.
Tandem found in frame 436.
Tandem found in frame 437.
Tandem found in frame 438.
Tandem found in frame 439.
Tandem found in frame 440.
Tandem found in frame 442.
Tandem found in frame 443.
Tandem found in frame 443.
Tandem found in frame 444.

Tandem found in frame 711.
Tandem found in frame 711.
Tandem found in frame 715.
Tandem found in frame 716.
Tandem found in frame 716.
Tandem found in frame 717.
Tandem found in frame 717.
Tandem found in frame 719.
Processed 24s of video. Time Elapsed 3725.526160955429s
Tandem found in frame 720.
Tandem found in frame 720.
Tandem found in frame 720.
Tandem found in frame 724.
Tandem found in frame 731.
Tandem found in frame 732.
Tandem found in frame 733.
Tandem found in frame 737.
Tandem found in frame 740.
Tandem found in frame 741.
Tandem found in frame 746.
Tandem found in frame 747.
Processed 25s of video. Time Elapsed 110.02437996864319s
Tandem found in frame 751.
Tandem found in frame 752.
Tandem found in frame 753.
Tandem found in frame 756.
Tandem found in frame 764.
Tandem found in frame 767.
Tandem found in frame 775.
Processed 26s of video. Time Elapsed 104.26413702964783s
Tandem found in frame 781.
Tandem found in frame 782.
Tandem found in frame 784.
Tandem found in fram

In [13]:
sess.close()

In [22]:
out_video.release()